In [97]:
import numpy as np
from Include.generateL1c import weil_sequence, bits_to_octal
import pandas as pd
from init_settings import init_settings

In [98]:
df = pd.read_csv("L1Cp.csv",
                 dtype={"PRN": int,
                        "w_pilot": int,
                        "p_pilot": int,
                        "Initial_pilot_code": str,
                        "Final_pilot_code":str
                       }
                )
L1c_table = df.set_index("PRN", verify_integrity=True)


In [99]:
PRN = 11
W1 = L1c_table.at[PRN, "w_pilot"]
P1 = L1c_table.at[PRN, "p_pilot"]
print (f"PRN = {PRN}, W={W1}, P={P1}")

PRN = 11, W=5093, P=253


In [100]:
ws = weil_sequence(W1, P1)
verification_string = bits_to_octal(ws)
print (f"Verification String {verification_string[0:12]}")

Verification String 047330766742


In [101]:
weil_bpsk = 2*ws-1
weil_complement = -1*weil_bpsk
BOC11 = np.column_stack((weil_bpsk, weil_complement)).ravel()
print (len(BOC11))
print (BOC11[0:10])

20460
[-1  1 -1  1 -1  1  1 -1 -1  1]


In [159]:
samplingFreq = 18E6
codeFreqBasis = 1.023E6
codeLength = len(ws)
codePeriod = codeLength / codeFreqBasis
tChip = 1 / codeFreqBasis
tSample = 1 / samplingFreq
sampleLength = tSample*codePeriod

#samples_per_code = int(round(samplingFreq / (codeFreqBasis / (codeLength - 1))))
#samples_per_code = samplingFreq / (codeFreqBasis / (codeLength - 1))
samples_per_code = round(codePeriod * samplingFreq)
samples_per_chip = samplingFreq / (2 * codeFreqBasis)   # Multiply by 2 because of BOC11; can be a float

#index =  np.round((1/samples_per_chip) * np.arange(0, samples_per_code)).astype(int) 
#index_float = ((2 * codeLength - 1)/ (samples_per_code -1)) * np.arange(0, samples_per_code)
index_float = (((2 * codeLength)/ (samples_per_code - 1)) * np.arange(0, samples_per_code) - 0.5)
index = (np.round(index_float)).astype(int)
index[-1] = codeLength-1   # just in case the round function rolled the last value out of bounds



#L1c_samples = BOC11[index]
#print (f"{L1c_samples[:20]}")

In [163]:
print (f'Code Length: {codeLength}')
print (f'Code Period: {codePeriod:.4f}')
print (f'samples per code: {samples_per_code}')
print (f'start of index_float: {index_float[:10]}')
print (f'end of index_float: {index_float[-10:]}')
print (f'start of index: {code_value_index[0:20]}')
print (f'end of index: {code_value_index[-20:]}')
print (f'BOC11: {BOC11[:10]}')

first = np.sum(code_value_index==0)
last = np.sum(code_value_index==10229)
one = np.sum(code_value_index==1)

print (f'first = {first}, last = {last}, one = {one}')


Code Length: 10230
Code Period: 0.0100
samples per code: 180000
start of index_float: [-0.5        -0.3863327  -0.2726654  -0.15899811 -0.04533081  0.06833649
  0.18200379  0.29567109  0.40933839  0.52300568]
end of index_float: [20458.47699432 20458.59066161 20458.70432891 20458.81799621
 20458.93166351 20459.04533081 20459.15899811 20459.2726654
 20459.3863327  20459.5       ]
start of index: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1]
end of index: [10228 10228 10228 10229 10229 10229 10229 10229 10229 10229 10229 10229
 10229 10229 10229 10229 10229 10229 10229 10229]
BOC11: [-1  1 -1  1 -1  1  1 -1 -1  1]
first = 17, last = 17, one = 18


In [162]:
def L1c_FS (prn: int, settings) -> np.ndarray:
    """
    Function receives:
        a prn number corresponding to an SV
        a set of settings for the data file & processing
    Returns:
        a numpy array with the L1 pilot signal (including BOC(1,1)
        at the sample rate of the data.  
        The L1c pilot signal will be 10mS long
    """
    df = pd.read_csv("L1Cp.csv",
                 dtype={"PRN": int,
                        "w_pilot": int,
                        "p_pilot": int,
                        "Initial_pilot_code": str,
                        "Final_pilot_code":str
                       }
                )
    L1c_table = df.set_index("PRN", verify_integrity=True)
    W1 = L1c_table.at[PRN, "w_pilot"]
    P1 = L1c_table.at[PRN, "p_pilot"]
    print (f'W1 = {W1}, P1={P1}')
    weil_code = weil_sequence(W1, P1)
    weil_bpsk = 2*ws-1               #re-map the weil code from binary to {+1, -1}
    weil_complement = -1*weil_bpsk   #invert the bpsk version
    weil_combined = np.column_stack((weil_bpsk, weil_complement))
    BOC11 = weil_combined.ravel()
    #
    #  Figure out the stretch factors to re-map the BOC11 to make it 10mS long
    #  At the sampling rate of the captured signal
    #
    codeLength = len(weil_code)
    codePeriod = 0.01   # Period of the L1c code in seconds
    samplingFreq = settings.samplingFreq
    samples_per_code = round((codePeriod * samplingFreq))
    #
    #
    index_float = ((2 * codeLength - 1)/ (samples_per_code - 1)) * np.arange(0, samples_per_code)
    index = (np.round(index_float)).astype(int)
    L1c_samples = BOC11[index]
    return (L1c_samples)

    
    

In [124]:
my_settings = init_settings()

In [125]:
L1c_pilot_signal = L1c_FS(1,my_settings)
print (L1c_pilot_signal.shape)
print (L1c_pilot_signal[:25])

W1 = 5093, P1=253
20460
(180000,)
[-1 -1 -1 -1 -1  1  1  1  1  1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1 -1  1  1
  1]


In [128]:
my_settings.acqSatelliteList = [1, 14, 21, 23]
print (my_settings.acqSatelliteList)

[1, 14, 21, 23]


In [137]:
y = np.load("acqResults.npy", allow_pickle=True).item()
print (y)

{'PRN': array([  1.,   0.,   0.,   0.,   0.,   6.,   7.,   0.,   0.,   0.,   0.,
         0.,  13.,  14.,   0.,   0.,  17.,   0.,  19.,   0.,  21.,   0.,
         0.,  24.,   0.,   0.,   0.,  28.,   0.,  30.,   0.,  87.,  88.,
         0.,   0.,   0.,   0.,  93.,  94.,   0.,   0.,   0.,   0.,   0.,
       100., 101.,   0.,   0., 104.,   0.,   0.]), 'carrFreq': array([17450.,     0.,     0.,     0.,     0., 23325., 15900.,     0.,
           0.,     0.,     0.,     0., 17000., 18475.,     0.,     0.,
       19000.,     0., 21000.,     0., 16500.,     0.,     0., 22400.,
           0.,     0.,     0., 19800.,     0., 16375.,     0., 19750.,
       17450.,     0.,     0.,     0.,     0., 23325., 15900.,     0.,
           0.,     0.,     0.,     0., 17000., 18475.,     0.,     0.,
       19000.,     0.,     0.]), 'codePhase': array([  275,     0,     0,     0,     0, 11389, 15293,     0,     0,
           0,     0,     0, 16477,  5954,     0,     0,   178,     0,
        1786,     0,  246

In [140]:
print (y.keys())
print (y['PRN'])
#print (x)

dict_keys(['PRN', 'carrFreq', 'codePhase', 'peakMetric'])
[  1.   0.   0.   0.   0.   6.   7.   0.   0.   0.   0.   0.  13.  14.
   0.   0.  17.   0.  19.   0.  21.   0.   0.  24.   0.   0.   0.  28.
   0.  30.   0.  87.  88.   0.   0.   0.   0.  93.  94.   0.   0.   0.
   0.   0. 100. 101.   0.   0. 104.   0.   0.]


In [134]:
print (y.keys())

AttributeError: 'numpy.ndarray' object has no attribute 'keys'

In [143]:
block_iii = [1, 4, 11, 14, 18, 21, 23, 28]
detected_sv = y['PRN']
detected_block_iii = (detected_sv[np.isin(detected_sv, block_iii)]).astype(int)
print (detected_block_iii)

[ 1 14 21 28]


In [145]:
for prn in detected_block_iii:
    print (i)
    x = 
    

1
14
21
28
